In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re # The RegEx module used for string operations

In [6]:
# Some font formatting constants:

FONT_BOLD = '\033[1m'
FONT_UNDERLINE = '\033[4m'
FONT_FORMAT_END = '\033[0m'

# Loading and observing the data

In [7]:
df = pd.read_csv('./billionaires.csv')

In [8]:
print(f"\nThe dataset consists of {FONT_BOLD}%d instances{FONT_FORMAT_END} and {FONT_BOLD}%d features{FONT_FORMAT_END}:\n{'─'*80}\n"
      %df.shape)

# Setting this option enables the presentation of all features instead of omitting the middle ones:
from IPython.display import display
with pd.option_context('display.max_columns', None):
    display(df.sample(10)) # Retrieves random instances from the dataset


The dataset consists of 2614 instances and 22 features:
────────────────────────────────────────────────────────────────────────────────



,name,rank,year,company.founded,company.name,company.relationship,company.sector,company.type,demographics.age,demographics.gender,location.citizenship,location.country code,location.gdp,location.region,wealth.type,wealth.worth in billions,wealth.how.category,wealth.how.from emerging,wealth.how.industry,wealth.how.inherited,wealth.how.was founder,wealth.how.was political
1134,Chairul Tanjung,375,2014,1987,Para Group (aka CT Corp),founder,investment banking,new,51,male,Indonesia,IDN,0.000000e+00,East Asia,privatized and resources,4.0,Financial,True,Diversified financial,not inherited,True,True
1294,Sydell Miller,421,2001,1980,Matrix,founder,hair care products,new,0,female,United States,USA,1.060000e+13,North America,founder non-finance,1.2,Traded Sectors,True,Consumer,not inherited,True,True
1974,M.A. Yusuff Ali,988,2014,1973,EMKE Group (Lulu Hypermarket),relation,groceries,new,58,male,India,IND,0.000000e+00,South Asia,executive,1.8,Non-Traded Sectors,True,"Retail, Restaurant",not inherited,True,True
471,Michele Ferrero,154,1996,1946,Ferrero spa,relation,chocolates,new,69,male,Italy,ITA,1.310000e+12,Europe,executive,2.3,Traded Sectors,True,Consumer,not inherited,True,True
2258,Bruce Cheng,1284,2014,1971,Delta Electronics,founder,electronics,new,78,male,Taiwan,Taiwan,0.000000e+00,East Asia,founder non-finance,1.3,New Sectors,True,Technology-Computer,not inherited,True,True
669,Charles Johnson,222,2001,1947,Franklin Resources,relation,mutal funds,new,68,male,United States,USA,1.060000e+13,North America,inherited,2.1,Financial,True,Money Management,father,True,True
1191,Adolf Merckle,388,1996,1881,Ratiopharm,relation,pharmaceuticals,new,61,male,Germany,DEU,2.500000e+12,Europe,inherited,1.0,New Sectors,True,Technology-Medical,4th generation,True,True
416,David Koch,138,2001,1940,Koch industries,relation,Oil refining,new,60,male,United States,USA,1.060000e+13,North America,inherited,3.2,Resource Related,True,Energy,father,True,True
886,Lawrence Flinn Jr,292,2001,1975,UVSG,investor,media,aquired,0,male,United States,USA,1.060000e+13,North America,executive,1.7,Non-Traded Sectors,True,Media,not inherited,True,True
869,Ayhan Sahenk,288,1996,1951,Dogus Holding,founder,construction,new,0,male,Turkey,TUR,1.810000e+11,Middle East/North Africa,self-made finance,1.4,Financial,True,Money Management,not inherited,True,True


## Features and their types

In [9]:
print(f"The {FONT_BOLD}string{FONT_FORMAT_END} and/or {FONT_BOLD}categorical{FONT_FORMAT_END} features include:\n{'─'*80}\n{df.select_dtypes(include=['object']).dtypes}\n")

print(f"The {FONT_BOLD}numerical{FONT_FORMAT_END} features include:\n{'─'*80}\n{df.select_dtypes(include=['int64', 'float64']).dtypes}\n")

print(f"There are also several {FONT_BOLD}boolean{FONT_FORMAT_END} features, including:\n{'─'*80}\n{df.select_dtypes(include=['bool']).dtypes}\n")

The string and/or categorical features include:
────────────────────────────────────────────────────────────────────────────────
name                     object
company.name             object
company.relationship     object
company.sector           object
company.type             object
demographics.gender      object
location.citizenship     object
location.country code    object
location.region          object
wealth.type              object
wealth.how.category      object
wealth.how.industry      object
wealth.how.inherited     object
dtype: object

The numerical features include:
────────────────────────────────────────────────────────────────────────────────
rank                          int64
year                          int64
company.founded               int64
demographics.age              int64
location.gdp                float64
wealth.worth in billions    float64
dtype: object

There are also several boolean features, including:
────────────────────────────────────────────

# Preprocessing
This includes all data cleanse measures, including:
<ul>
    <li>Identification and correction of spelling errors</li>
    <li>Making common categories (values in categorical features) consistent</li>
    <li>Filling in missing data</li>
    <li>Removing duplicates</li>
    <li>Inspecting outliers</li>
    <li>Casting features to the suitable types according to the data that they contain</li>
</ul>

# Cleaning up features

In [10]:
from difflib import SequenceMatcher

# The following function receives two lists of words and maps the similar ones together;
# It is used to clean up categorical features by pinpointing the 'almost' similar words and
# consequently identifying the spelling errors.
def similar(word_list):
    for i in range(len(word_list)):
        first_word = word_list[i]
        if pd.isna(first_word):
            continue
        print(first_word, '\n', '-'*30)
        res = []
        for j in range(i+1, len(word_list)):
            second_word = word_list[j]
            if pd.isna(second_word):
                continue
            if SequenceMatcher(None, first_word, second_word).ratio() > .7:
                res = res + [second_word]
        if len(res) > 0:
            print(res, '\n\n')

## Common touches
These refactorings are applicable to all string features and they include:
<ul>
    <li>Removing trailing and leading spaces from names</li>
    <li>Removing all consecutive spaces in the middle of the names</li>
    <li>Removing all extra dots</li>
    <li>Capitalizing the first letter of each word</li>
</ul>

In [11]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].apply(
        lambda x: x.strip()
                   .replace('..', '.')
                   .replace('  ', ' ')
                   .title()
                  if isinstance(x, str) else x
    )

## Processing the Boolean features
All of these features contain only a single value (True), and as a result, they do not mandate any data cleanse tasks.

### wealth.how.from emerging

In [12]:
df['wealth.how.from emerging'].unique()

array([ True])

### wealth.how.was political

In [13]:
df['wealth.how.was political'].unique()

array([ True])

### wealth.how.was founder

In [14]:
df['wealth.how.was founder'].unique()

array([ True])

## Processing the categorical features
These features generally need the following retouches:
<ul>
    <li>Correcting the spelling errors and typos</li>
    <li>Filling in the missing values (NAs) with 'Unspecified'</li>
    <li>Filling in the value '0' with 'Unspecified'</li>
    <li>Casting the variable from string (object) to categorical</li>
</ul>
For some of these variables, one or more of the mentioned tasks might not be necessary.

### wealth.how.inherited

In [15]:
df['wealth.how.inherited'].value_counts(dropna=False)

Not Inherited               1688
Father                       558
3Rd Generation               210
4Th Generation                68
Spouse/Widow                  59
5Th Generation Or Longer      31
Name: wealth.how.inherited, dtype: int64

In [16]:
# Casting the feature into categorical type
df['wealth.how.inherited'] = df['wealth.how.inherited'].astype('category')

df['wealth.how.inherited'].dtype

CategoricalDtype(categories=['3Rd Generation', '4Th Generation',
                  '5Th Generation Or Longer', 'Father', 'Not Inherited',
                  'Spouse/Widow'],
, ordered=False)

### wealth.how.industry

In [17]:
df['wealth.how.industry'].value_counts(dropna=False)

Consumer                           471
Retail, Restaurant                 281
Real Estate                        280
Money Management                   249
Media                              219
Technology-Computer                208
Diversified Financial              167
Energy                             132
Technology-Medical                 111
Non-Consumer Industrial            107
Constrution                         97
Mining And Metals                   90
Other                               83
Hedge Funds                         67
Private Equity/Leveraged Buyout     25
0                                   16
Venture Capital                      8
Banking                              1
Services                             1
NaN                                  1
Name: wealth.how.industry, dtype: int64

In [18]:
# Replacing the value of '0' with 'Unspecified'
df['wealth.how.industry'] = df['wealth.how.industry'].replace(
    {
        '0': 'Unspecified',
    }
)

# Filling Null values with the value 'Unspecified'
df['wealth.how.industry'] = df['wealth.how.industry'].fillna('Unspecified')

# Casting the feature into categorical type
df['wealth.how.industry'] = df['wealth.how.industry'].astype('category')

df['wealth.how.industry'].dtype

CategoricalDtype(categories=['Banking', 'Constrution', 'Consumer',
                  'Diversified Financial', 'Energy', 'Hedge Funds', 'Media',
                  'Mining And Metals', 'Money Management',
                  'Non-Consumer Industrial', 'Other',
                  'Private Equity/Leveraged Buyout', 'Real Estate',
                  'Retail, Restaurant', 'Services', 'Technology-Computer',
                  'Technology-Medical', 'Unspecified', 'Venture Capital'],
, ordered=False)

### wealth.how.category

In [19]:
df['wealth.how.category'].value_counts(dropna=False)

Financial             800
Non-Traded Sectors    597
Traded Sectors        564
New Sectors           319
Resource Related      245
0                      85
NaN                     1
Finance                 1
Trucking                1
Energy                  1
Name: wealth.how.category, dtype: int64

In [20]:
# Replacing the value of '0' with 'Unspecified'
df['wealth.how.category'] = df['wealth.how.category'].replace(
    {
        '0': 'Unspecified',
    }
)

# Filling Null values with the value 'Unspecified'
df['wealth.how.category'] = df['wealth.how.category'].fillna('Unspecified')

# Casting the feature into categorical type
df['wealth.how.category'] = df['wealth.how.category'].astype('category')

df['wealth.how.category'].dtype

CategoricalDtype(categories=['Energy', 'Finance', 'Financial', 'New Sectors',
                  'Non-Traded Sectors', 'Resource Related', 'Traded Sectors',
                  'Trucking', 'Unspecified'],
, ordered=False)

### wealth.type

In [21]:
df['wealth.type'].value_counts(dropna=False)

Inherited                   953
Founder Non-Finance         713
Self-Made Finance           500
Privatized And Resources    236
Executive                   190
NaN                          22
Name: wealth.type, dtype: int64

In [22]:
# Filling Null values with the value 'Unspecified'
df['wealth.type'] = df['wealth.type'].fillna('Unspecified')

# Casting the feature into categorical type
df['wealth.type'] = df['wealth.type'].astype('category')

df['wealth.type'].dtype

CategoricalDtype(categories=['Executive', 'Founder Non-Finance', 'Inherited',
                  'Privatized And Resources', 'Self-Made Finance',
                  'Unspecified'],
, ordered=False)

### location.region

In [23]:
df['location.region'].value_counts(dropna=False)

North America               992
Europe                      698
East Asia                   535
Latin America               182
Middle East/North Africa    117
South Asia                   69
Sub-Saharan Africa           20
0                             1
Name: location.region, dtype: int64

In [24]:
df['location.region'] = df['location.region'].replace(
    {
        '0': 'Unspecified'
    }
)

# Casting the feature into categorical type
df['location.region'] = df['location.region'].astype('category')

df['location.region'].dtype

CategoricalDtype(categories=['East Asia', 'Europe', 'Latin America',
                  'Middle East/North Africa', 'North America', 'South Asia',
                  'Sub-Saharan Africa', 'Unspecified'],
, ordered=False)

### location.country code

In [25]:
df['location.country code'].value_counts(dropna=False)

Usa    903
Deu    160
Chn    153
Rus    119
Jpn     96
      ... 
Bmu      1
Swz      1
Ago      1
Bhr      1
Tza      1
Name: location.country code, Length: 74, dtype: int64

In [26]:
# Casting the feature into categorical type
df['location.country code'] = df['location.country code'].astype('category')

df['location.country code'].dtype

CategoricalDtype(categories=['Ago', 'Are', 'Arg', 'Aus', 'Aut', 'Bel', 'Bhr', 'Bmu',
                  'Bra', 'Can', 'Che', 'Chl', 'Chn', 'Col', 'Cyp', 'Cze',
                  'Den', 'Deu', 'Dnk', 'Dza', 'Ecu', 'Egy', 'Esp', 'Fin',
                  'Fra', 'Gbr', 'Geo', 'Ggy', 'Grc', 'Hkg', 'Idn', 'Ind',
                  'Irl', 'Isr', 'Ita', 'Jpn', 'Kaz', 'Kna', 'Kor', 'Kwt',
                  'Lbn', 'Lie', 'Ltu', 'Mac', 'Mar', 'Mco', 'Mex', 'Mys',
                  'Nga', 'Nld', 'Nor', 'Npl', 'Nzl', 'Omn', 'Per', 'Phl',
                  'Pol', 'Prt', 'Rou', 'Rus', 'Sau', 'Sgp', 'Swe', 'Swz',
                  'Taiwan', 'Tha', 'Tur', 'Tza', 'Uga', 'Ukr', 'Usa', 'Ven',
                  'Vnm', 'Zaf'],
, ordered=False)

### location.citizenship

In [27]:
df['location.citizenship'].value_counts(dropna=False)

United States    903
Germany          160
China            153
Russia           119
Japan             96
                ... 
Bahrain            1
Ecuador            1
Georgia            1
Bermuda            1
Tanzania           1
Name: location.citizenship, Length: 73, dtype: int64

In [28]:
# Casting the feature into categorical type
df['location.citizenship'] = df['location.citizenship'].astype('category')

df['location.citizenship'].dtype

CategoricalDtype(categories=['Algeria', 'Angola', 'Argentina', 'Australia', 'Austria',
                  'Bahrain', 'Belgium', 'Bermuda', 'Brazil', 'Canada', 'Chile',
                  'China', 'Colombia', 'Cyprus', 'Czech Republic', 'Denmark',
                  'Ecuador', 'Egypt', 'Finland', 'France', 'Georgia',
                  'Germany', 'Greece', 'Guernsey', 'Hong Kong', 'India',
                  'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan',
                  'Kazakhstan', 'Kuwait', 'Lebanon', 'Liechtenstein',
                  'Lithuania', 'Macau', 'Malaysia', 'Mexico', 'Monaco',
                  'Morocco', 'Nepal', 'Netherlands', 'New Zealand', 'Nigeria',
                  'Norway', 'Oman', 'Peru', 'Philippines', 'Poland',
                  'Portugal', 'Romania', 'Russia', 'Saudi Arabia', 'Singapore',
                  'South Africa', 'South Korea', 'Spain',
                  'St. Kitts And Nevis', 'Swaziland', 'Sweden', 'Switzerland',
                  'Taiwan', 'Tanzan

### demographics.gender

In [ ]:
df['demographics.gender'].value_counts()

In [ ]:
df['demographics.gender'].isna().sum()

In [ ]:
df['demographics.gender'] = df['demographics.gender'].fillna('Unspecified')

### company.type

In [ ]:
df['company.type'].sort_values().unique()

In [ ]:
df['company.type'] = df['company.type'].replace(
    {
        'aquired': 'acquired',
        'franchise rights': 'franchise',
        'neew': 'new',
        'new division': 'new',
        'new, privitization': 'new/privatized',
        'privatization': 'privatized'
    }
)

In [ ]:
df['company.type'].isna().sum()

In [ ]:
df['company.type'] = df['company.type'].fillna('Unspecified')

### company.sector

In [ ]:
df['company.sector'].sort_values().unique()

In [ ]:

df['company.sector'] = df['company.sector'].replace(
    {
        'Banking': 'banking',
        'Communications': 'communications',
        'Fashion': 'fashion',
        'Finance': 'finance',
        'Oil refining': 'oil refining',
        'Software': 'software',
        'agriculture': 'agricultural',
        'agriculteral': 'agricultural',
        'aigriculture': 'agricultural',
        'airports': 'airport',
        'aluminum, oil': 'aluminum/oil',
        'aplliances': 'appliances',
        'automobile dealers': 'auto dealerships',
        'automotive parts': 'auto parts',
        'automotive components': 'auto components',
        'automotive': 'autos',
        'automotives': 'autos',
        'bank': 'banking',
        'banking and insurance': 'banking/insurance',
        'banking, real estate': 'banking/real estate',
        'banking and real estate': 'banking/real estate',
        'banking, media': 'banking/media',
        'banking, oil, aluminum': 'banking/oil/aluminum',
        'braking systems': 'brake systems',
        'cars, finance': 'cars/finance',
        'cigarette': 'cigarettes',
        'comodities': 'commodities',
        'commodity trading': 'commodities trading',
        'construcion': 'construction',
        'construction, hotels, gaming': 'construction/hotels/gaming',
        'construction, utilities': 'construction/utilities',
        'construction,banking': 'construction/banking',
        'copper, poultry': 'copper/poultry',
        'e-commerce, venture capital': 'e-commerce/venture capital',
        'education, language schools': 'education/language schools',
        'electonics': 'electronics',
        'electonics components': 'electronic components',
        'energy, retail, manufacturing': 'energy/retail/manufacturing',
        'property, textile manufacturing': 'property/textile manufacturing',
        'eyeglasses': 'glasses',
        'fertalizers': 'fertilizers',
        'fertilizer': 'fertilizers',
        'finace': 'finance',
        'forestry and mining': 'forestry/mining',
        'metals and mining': 'metals/mining',
        'gambing': 'gambling',
        'media, pipelines': 'media/pipelines',
        'gas, petrochemicals': 'gas/petrochemicals',
        'internet companies': 'internet company',
        'internet provider': 'internet service provider',
        'investments': 'investment',
        'invetsments': 'investment',
        'investments/real estate, sports teams': 'investments/real estate/sports teams',
        'iron and steel': 'iron/steel',
        'leveraged buyout': 'leveraged buyouts',
        'lumber, real estate': 'lumber/real estate',
        'manufacturing, real estate, aviation': 'manufacturing/real estate/aviation',
        'mining, investments': 'mining/investments',
        'mining, steel': 'mining/steel',
        'mutal funds': 'mutual funds',
        'oil and gas': 'oil/gas',
        'oil and natural gas': 'oil/gas',
        'oil, railroad, investments': 'oil/railroad/investments',
        'oil field services': 'oilfield services',
        'petrochemicals and textiles': 'petrochemicals/textiles',
        'petrochemicals, textiles': 'petrochemicals/textiles',
        'plastic': 'plastics',
        'ports, gas': 'ports/gas',
        'postal service, beverages': 'postal service/beverages',
        'real estate and metals': 'real estate/metals',
        'real estate, financial services and computers': 'real estate/financial services/computers',
        'real estate, retail': 'real estate/retail',
        'restaurant': 'restaurants',
        'retail, financial services, real estate': 'retail/financial services/real estate',
        'retail, home appliances': 'retail/home appliances',
        'rice trading, mining, infrastructure': 'rice trading/mining/infrastructure',
        'self storage': 'self-storage',
        'stock brokerage': 'stock broker',
        'sugar, flour, cement': 'sugar/flour/cement',
        'telecom, finance': 'telecom/finance',
        'telecom, oil, beer': 'telecom/oil/beer',
        'telecomm': 'telecom',
        'textiles, industrial goods, media': 'textiles/industrial goods/media',
        'timber and paper': 'timber/paper',
        'timber, plantations': 'timber/plantations',
        'trading company': 'trading',
        'truck stop': 'truck stops',
        'trucking and logistics': 'trucking/logistics',
        'apparel and eyewear': 'apparel/eyewear',
        'brokerage/banking': 'broker/banking',
        'chocolate and coffee': 'chocolates/coffee',
        'electronic componants': 'electronic components',
        'elevators, escalators, engineering': 'elevators/escalators/engineering',
        'food and energy processess': 'food and energy processes',
        'food, entertainment': 'food/entertainment',
        'gas and propane': 'gas/propane',
        'hospitality and food service': 'hospitality/food service',
        'hotels and supermarkets': 'hotels/supermarkets',
        'industrial manufacturing and banking': 'industrial manufacturing/banking',
        'insurancei, power': 'insurance/power',
        'media, exports': 'media/exports',
        'metals, paper, cement': 'metals/paper/cement',
        'mining, banking': 'mining/banking',
        'oleochemicals and palm oil': 'oleochemicals/palm oil',
        'palm oil, mining': 'palm oil/mining',
        'pet supplies, real estate': 'pet supplies/real estate',
        'printing and electronics': 'printing/electronics'
    }
)

### company.relationship

In [ ]:
df['company.relationship'].sort_values().unique()

In [ ]:
similar(df["company.relationship"].sort_values().unique())

### name

Listing possible inconsistancies where the names are different, but the following features are the same:
<ul>
    <li>company.founded</li>
    <li>company.name</li>
    <li>company.relationship</li>
    <li>company.sector</li>
    <li>company.type</li>
    <li>demographics.gender</li>
    <li>location.citizenship</li>
    <li>location.country code</li>
    <li>location.region</li>
    <li>wealth.type</li>
    <li>wealth.how.category</li>
    <li>wealth.how.from emerging</li>
    <li>wealth.how.industry</li>
    <li>wealth.how.inherited</li>
</ul>

In [ ]:
dup_cols = ['company.founded',
            'company.name',
            'company.relationship',
            'company.sector',
            'company.type',
            'demographics.gender',
            'location.citizenship',
            'location.country code',
            'location.region',
            'wealth.type',
            'wealth.how.category',
            'wealth.how.from emerging',
            'wealth.how.industry',
            'wealth.how.inherited']

dup_rows = df[df.duplicated(subset=dup_cols)]
dup_rows[duprows[]]

In [ ]:
import re

df[df['name'].apply(lambda x: (re.search('abraham', x, re.IGNORECASE) is not None))]

In [ ]:
import re

df[df['name'].apply(lambda x: (re.search('abraham', x, re.IGNORECASE) is not None))]